# eFAST Interactive Runner

Select a site and season, then run eFAST processing and visualization.

**Setup:**
```bash
pip install ipywidgets matplotlib rasterio numpy
jupyter nbextension enable --py widgetsnbextension
```

In [ ]:
import json
from pathlib import Path
from datetime import datetime, timedelta
from dateutil.rrule import rrule, DAILY
from IPython.display import display
import ipywidgets as widgets

try:
    import rasterio
    import matplotlib.pyplot as plt
    import numpy as np
    RASTERIO_AVAILABLE = True
except ImportError:
    RASTERIO_AVAILABLE = False
    print("⚠️ Install rasterio for visualization: pip install rasterio matplotlib numpy")

try:
    import efast
    EFAST_AVAILABLE = True
except ImportError:
    EFAST_AVAILABLE = False
    print("⚠️ Install efast package: pip install -e .")

try:
    import run_efast
    RUN_EFAST_AVAILABLE = True
except ImportError:
    RUN_EFAST_AVAILABLE = False


## 1. Site Selection

In [36]:
# Load sites
with open('selected_sites.geojson', 'r') as f:
    geojson_data = json.load(f)

sites = {}
for feature in geojson_data['features']:
    props = feature['properties']
    sitename = props['sitename']
    sites[sitename] = {
        'coordinates': feature['geometry']['coordinates'],
        'description': props.get('description', ''),
        'seasons': props.get('seasons', {})
    }

# Create widgets
site_dropdown = widgets.Dropdown(options=sorted(sites.keys()), description='Site:', style={'description_width': '100px'})
season_dropdown = widgets.Dropdown(options=[], description='Season:', style={'description_width': '100px'})
info_output = widgets.Output()

def update_season(change=None):
    if site_dropdown.value:
        seasons = sorted(sites[site_dropdown.value]['seasons'].keys(), reverse=True)
        season_dropdown.options = seasons
        if seasons:
            season_dropdown.value = seasons[0] if '2024' not in seasons else '2024'
    
    with info_output:
        info_output.clear_output()
        if site_dropdown.value and season_dropdown.value:
            site = sites[site_dropdown.value]
            season_data = site['seasons'][season_dropdown.value]
            print(f"📍 {site_dropdown.value}")
            print(f"📅 {season_data['season_start_date']} to {season_data['season_end_date']}")
            print(f"🛰️ S2: {season_data['sentinel2_scenes']} scenes | S3: {season_data['sentinel3_scenes']} scenes")

site_dropdown.observe(update_season, names='value')

# Initialize
if 'innsbruck' in site_dropdown.options:
    site_dropdown.value = 'innsbruck'
else:
    site_dropdown.value = site_dropdown.options[0]
update_season()

display(widgets.VBox([
    widgets.HTML("<h3>Select Site and Season</h3>"),
    widgets.HBox([site_dropdown, season_dropdown]),
    info_output
]))

## 2. Get Data

Download S2 and S3 data from CDSE, or use cached data if available.


In [ ]:
# Check data availability and download
test_data_dir = Path('test_data')
s2_raw_dir = test_data_dir / 'S2' / 'raw'
s3_raw_dir = test_data_dir / 'S3' / 'raw'

# Ensure RUN_EFAST_AVAILABLE is defined (fallback if imports cell wasn't run)
if 'RUN_EFAST_AVAILABLE' not in globals():
    try:
        import run_efast
        RUN_EFAST_AVAILABLE = True
    except ImportError:
        RUN_EFAST_AVAILABLE = False

# Get selected site/season
sitename = site_dropdown.value if 'site_dropdown' in globals() else None
season_year = season_dropdown.value if 'season_dropdown' in globals() else None
start_date = None
end_date = None

if sitename and season_year and sitename in sites:
    if season_year in sites[sitename]['seasons']:
        season_data = sites[sitename]['seasons'][season_year]
        start_date = season_data.get('season_start_date')
        end_date = season_data.get('season_end_date')

# Check cache
s2_files = list(s2_raw_dir.glob('*.SAFE')) if s2_raw_dir.exists() else []
s3_files = list(s3_raw_dir.glob('*.zip')) + list(s3_raw_dir.glob('*.nc')) if s3_raw_dir.exists() else []

download_output = widgets.Output()
download_button = widgets.Button(
    description='📥 Download Data',
    button_style='primary',
    disabled=not (RUN_EFAST_AVAILABLE and start_date and end_date)
)

def run_download(b):
    with download_output:
        download_output.clear_output()
        if not start_date or not end_date:
            print("❌ Select site and season first")
            return
        
        if s2_files and s3_files:
            print(f"✅ Data already cached: S2={len(s2_files)} files, S3={len(s3_files)} files")
            print("💡 Click again to re-download")
            return
        
        print(f"📥 Downloading data for {start_date} to {end_date}...")
        
        coords = sites[sitename]['coordinates']
        aoi_geometry = f"POINT ({coords[0]} {coords[1]})"
        
        try:
            credentials = run_efast.get_credentials_from_env()
        except:
            print("❌ CDSE credentials not found. Set CDSE_USERNAME and CDSE_PASSWORD")
            return
        
        s2_raw_dir.mkdir(parents=True, exist_ok=True)
        s3_raw_dir.mkdir(parents=True, exist_ok=True)
        
        try:
            run_efast.download_data(
                start_date, end_date, aoi_geometry,
                s2_raw_dir, s3_raw_dir, credentials, data_source="cdse"
            )
            print(f"✅ Download complete!")
            print(f"💡 Re-run this cell to refresh status")
        except Exception as e:
            print(f"❌ Error: {e}")
            import traceback
            traceback.print_exc()

download_button.on_click(run_download)

print("=" * 60)
print("📊 DATA STATUS")
print("=" * 60)
print(f"S2 Raw: {'✅' if s2_files else '❌'} {len(s2_files)} files")
print(f"S3 Raw: {'✅' if s3_files else '❌'} {len(s3_files)} files")
print("=" * 60)

display(widgets.VBox([download_button, download_output]))

# Load data for verification cells (runs silently - status shown in individual step cells)
if RASTERIO_AVAILABLE and test_data_dir.exists():
    # Load fusion files
    fusion_dir = test_data_dir / 'fusion_results'
    fusion_files = sorted(fusion_dir.glob('REFL_*.tif')) if fusion_dir.exists() else []
    
    # Filter by date range
    filtered_fusion = []
    if start_date and end_date and fusion_files:
        for f in fusion_files:
            try:
                date_str = f.stem.split('REFL_')[1].split('_')[0]
                if len(date_str) == 8:
                    file_date = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
                    if start_date <= file_date <= end_date:
                        filtered_fusion.append(f)
            except (IndexError, ValueError):
                pass
    else:
        filtered_fusion = fusion_files
    
    # Load S2 and S3 files
    s2_dir = test_data_dir / 'S2' / 'processed'
    s2_files = sorted(s2_dir.glob('*REFL.tif')) if s2_dir.exists() else []
    
    s3_composites_dir = test_data_dir / 'S3' / 'composites'
    s3_composites = sorted(s3_composites_dir.glob('composite*.tif')) if s3_composites_dir.exists() else []
else:
    # Initialize empty if not available
    filtered_fusion = []
    s2_files = []
    s3_composites = []

📊 DATA STATUS
S2 Raw: ❌ 0 files
S3 Raw: ❌ 0 files


📊 EFAST PIPELINE STATUS
Site: innsbruck | Season: 2024
Date range: 2024-01-01 to 2024-12-31

1️⃣  DOWNLOAD
   S2 Raw: ❌ 0 files
   S3 Raw: ❌ 0 files

2️⃣  S2 PROCESSING
   Processed: ✅ 1 files

3️⃣  S3 PROCESSING
   Binning: ✅ 365 files
   Composites: ✅ 183 files
   Reprojected: ✅ 183 files

4️⃣  FUSION
   Results: ❌ 0 files


📂 Loading data for verification...
✅ Loaded:
   Fusion files: 0
   S2 files: 1
   S3 composites: 183


## 3. Processing

Process S2 (extract and mask bands) and S3 (bin, composite, smooth, reproject) data. Uses cached results if available.


In [38]:
# Check processing status
s2_processed_dir = test_data_dir / 'S2' / 'processed'
s3_reprojected_dir = test_data_dir / 'S3' / 'reprojected'

s2_processed = list(s2_processed_dir.glob('*REFL.tif')) if s2_processed_dir.exists() else []
s3_reprojected = list(s3_reprojected_dir.glob('composite*.tif')) if s3_reprojected_dir.exists() else []

process_output = widgets.Output()
process_button = widgets.Button(
    description='⚙️ Process Data',
    button_style='info',
    disabled=not (EFAST_AVAILABLE and s2_files and s3_files)
)

def run_processing(b):
    with process_output:
        process_output.clear_output()
        
        if not s2_files or not s3_files:
            print("❌ Download data first")
            return
        
        if s2_processed and s3_reprojected:
            print(f"✅ Processing already done: S2={len(s2_processed)} files, S3={len(s3_reprojected)} files")
            print("💡 Click again to re-process")
            return
        
        print("⚙️ Processing data...")
        
        try:
            import efast.s2_processing as s2
            import efast.s3_processing as s3
            
            # S2 Processing
            print("\n🛰️ Processing S2...")
            s2_processed_dir.mkdir(parents=True, exist_ok=True)
            s2.extract_mask_s2_bands(s2_raw_dir, s2_processed_dir, bands=["B02", "B03", "B04", "B8A"])
            s2.distance_to_clouds(s2_processed_dir, ratio=30)
            
            # S3 Processing
            print("\n🌊 Processing S3...")
            footprint = s2.get_wkt_footprint(s2_processed_dir)
            
            s3_binning_dir = test_data_dir / 'S3' / 'binning'
            s3_composites_dir = test_data_dir / 'S3' / 'composites'
            s3_blurred_dir = test_data_dir / 'S3' / 'blurred'
            s3_calibrated_dir = test_data_dir / 'S3' / 'calibrated'
            
            for d in [s3_binning_dir, s3_composites_dir, s3_blurred_dir, s3_calibrated_dir, s3_reprojected_dir]:
                d.mkdir(parents=True, exist_ok=True)
            
            s3.binning_s3(s3_raw_dir, s3_binning_dir, footprint=footprint,
                        s3_bands=["SDR_Oa04", "SDR_Oa06", "SDR_Oa08", "SDR_Oa17"],
                        instrument="OL", aggregator="mean")
            s3.produce_median_composite(s3_binning_dir, s3_composites_dir, mosaic_days=100, step=2)
            s3.smoothing(s3_composites_dir, s3_blurred_dir, std=1, preserve_nan=False)
            s3.reformat_s3(s3_blurred_dir, s3_calibrated_dir)
            s3.reproject_and_crop_s3(s3_calibrated_dir, s2_processed_dir, s3_reprojected_dir)
            
            print(f"\n✅ Processing complete!")
            print(f"💡 Re-run this cell to refresh status")
        except Exception as e:
            print(f"❌ Error: {e}")
            import traceback
            traceback.print_exc()

process_button.on_click(run_processing)

print("=" * 60)
print("📊 PROCESSING STATUS")
print("=" * 60)
print(f"S2 Processed: {'✅' if s2_processed else '❌'} {len(s2_processed)} files")
print(f"S3 Reprojected: {'✅' if s3_reprojected else '❌'} {len(s3_reprojected)} files")
print("=" * 60)

display(widgets.VBox([process_button, process_output]))


📊 PROCESSING STATUS
S2 Processed: ✅ 1 files
S3 Reprojected: ✅ 183 files


## 4. Fusion

Combine S2 and S3 into high-resolution daily images. Uses cached results if available.

In [ ]:
# Check fusion status
fusion_dir = test_data_dir / 'fusion_results'
fusion_files = sorted(fusion_dir.glob('REFL_*.tif')) if fusion_dir.exists() else []

# Use filtered_fusion from Cell 5 if available, otherwise count all files
if 'filtered_fusion' in globals():
    fusion_count = len(filtered_fusion)
else:
    fusion_count = len(fusion_files)

fusion_output = widgets.Output()
fusion_button = widgets.Button(
    description='🚀 Run Fusion',
    button_style='success',
    disabled=not (EFAST_AVAILABLE and s2_processed and s3_reprojected and start_date and end_date)
)

def run_fusion(b):
    with fusion_output:
        fusion_output.clear_output()
        
        if not s2_processed or not s3_reprojected:
            print("❌ Process S2 and S3 data first")
            return
        
        if not start_date or not end_date:
            print("❌ Select site and season first")
            return
        
        if fusion_count > 0:
            print(f"✅ Fusion already done: {fusion_count} files")
            print("💡 Click again to re-run fusion")
            return
        
        print(f"🚀 Running fusion for {start_date} to {end_date}...")
        
        try:
            fusion_dir.mkdir(parents=True, exist_ok=True)
            
            step = 2
            ratio = 30
            max_days = 15
            minimum_acquisition_importance = 0
            
            dates = list(rrule(
                DAILY,
                dtstart=datetime.strptime(start_date, '%Y-%m-%d') + timedelta(step),
                until=datetime.strptime(end_date, '%Y-%m-%d') - timedelta(step),
                interval=step,
            ))
            
            print(f"   Processing {len(dates)} dates...")
            print("   (This may take a while - suppressing verbose output)\n")
            
            import sys
            import io
            from tqdm import tqdm
            
            # Suppress verbose output from efast library
            class SuppressOutput:
                def __init__(self):
                    self.stdout = io.StringIO()
                    self.stderr = io.StringIO()
                
                def __enter__(self):
                    self.old_stdout = sys.stdout
                    self.old_stderr = sys.stderr
                    sys.stdout = self.stdout
                    sys.stderr = self.stderr
                    return self
                
                def __exit__(self, *args):
                    sys.stdout = self.old_stdout
                    sys.stderr = self.old_stderr
            
            successful = 0
            failed = 0
            error_dates = []
            
            # Process dates with progress bar
            for date in tqdm(dates, desc="Fusion progress", unit="date", ncols=80):
                try:
                    # Suppress verbose output from efast.fusion
                    with SuppressOutput():
                        efast.fusion(
                            date,
                            s3_reprojected_dir,
                            s2_processed_dir,
                            fusion_dir,
                            product="REFL",
                            ratio=ratio,
                            max_days=max_days,
                            minimum_acquisition_importance=minimum_acquisition_importance,
                        )
                    successful += 1
                except Exception as e:
                    failed += 1
                    error_dates.append((date.date(), str(e)))
                    # Only show first few errors to avoid clutter
                    if failed <= 5:
                        error_msg = str(e).split('\n')[0]  # Get first line only
                        print(f"\n   ⚠️ Error for {date.date()}: {error_msg[:80]}")
            
            # Summary of errors
            if failed > 5:
                print(f"\n   ... and {failed - 5} more errors (see summary below)")
            
            print(f"\n✅ Fusion complete!")
            print(f"   Successful: {successful}/{len(dates)} ({successful/len(dates)*100:.1f}%)")
            if failed > 0:
                print(f"   Failed: {failed}/{len(dates)} ({failed/len(dates)*100:.1f}%)")
                if failed <= 10:
                    print(f"\n   Failed dates:")
                    for date, error in error_dates:
                        error_short = error.split('\n')[0][:60]
                        print(f"     - {date}: {error_short}")
                else:
                    print(f"\n   First 10 failed dates:")
                    for date, error in error_dates[:10]:
                        error_short = error.split('\n')[0][:60]
                        print(f"     - {date}: {error_short}")
                    print(f"     ... and {failed - 10} more")
            print(f"\n💡 Re-run this cell to refresh status")
        except Exception as e:
            print(f"❌ Error: {e}")
            import traceback
            traceback.print_exc()

fusion_button.on_click(run_fusion)

print("=" * 60)
print("📊 FUSION STATUS")
print("=" * 60)
print(f"Fusion Results: {'✅' if fusion_count > 0 else '❌'} {fusion_count} files")
if start_date and end_date:
    print(f"Date range: {start_date} to {end_date}")
print("=" * 60)

display(widgets.VBox([fusion_button, fusion_output]))

📊 FUSION STATUS
Fusion Results: ❌ 0 files
Date range: 2024-01-01 to 2024-12-31


## 5. Verify Results

Three core visualizations for verification:
1. **Basic Statistics** - Data validity and consistency
2. **RGB Comparison** - Spatial quality (S2 vs S3 vs Fusion)
3. **Time Series** - Temporal smoothness


### 5.1 Basic Statistics


In [ ]:
# 1. Basic Statistics - Data validity and consistency

if not RASTERIO_AVAILABLE:
    print("⚠️ Install rasterio for visualization")
elif 'filtered_fusion' not in globals():
    print("❌ Variables not found. Run the data loading cell (Cell 5) first.")
elif not filtered_fusion:
    print("⚠️  No fusion files found!")
    print(f"   The fusion_results directory exists but is empty.")
    print(f"   💡 You need to run the EFAST fusion step first.")
    print(f"   Run: python run_efast.py (or use the fusion function)")
    print(f"\n   Available data:")
    print(f"   - S2 files: {len(s2_files) if 's2_files' in globals() else 0}")
    print(f"   - S3 composites: {len(s3_composites) if 's3_composites' in globals() else 0}")
    print(f"\n   You can still verify S2 and S3 data, but fusion results are needed for full verification.")
else:
    print("=" * 70)
    print("1️⃣  BASIC STATISTICS")
    print("=" * 70)
    
    stats_data = []
    for fusion_file in filtered_fusion[:10]:  # Sample first 10 files
        try:
            with rasterio.open(fusion_file) as src:
                data = src.read()
                date_str = fusion_file.stem.split('_')[1]
                
                file_stats = {
                    'date': date_str,
                    'shape': data.shape,
                    'bands': data.shape[0],
                    'valid_pixels': [],
                    'min': [],
                    'max': [],
                    'mean': [],
                    'std': []
                }
                
                for band_idx in range(data.shape[0]):
                    band_data = data[band_idx].astype(float)
                    valid = ~np.isnan(band_data)
                    valid_count = np.sum(valid)
                    
                    if valid_count > 0:
                        valid_data = band_data[valid]
                        file_stats['valid_pixels'].append(valid_count)
                        file_stats['min'].append(np.min(valid_data))
                        file_stats['max'].append(np.max(valid_data))
                        file_stats['mean'].append(np.mean(valid_data))
                        file_stats['std'].append(np.std(valid_data))
                    else:
                        file_stats['valid_pixels'].append(0)
                        file_stats['min'].append(np.nan)
                        file_stats['max'].append(np.nan)
                        file_stats['mean'].append(np.nan)
                        file_stats['std'].append(np.nan)
                
                stats_data.append(file_stats)
                print(f"📅 {date_str}: {data.shape[1]}x{data.shape[2]} pixels, {data.shape[0]} bands")
                print(f"   Band 1 (Red):   min={file_stats['min'][0]:.4f}, max={file_stats['max'][0]:.4f}, mean={file_stats['mean'][0]:.4f}")
                if len(file_stats['min']) > 1:
                    print(f"   Band 2 (Green): min={file_stats['min'][1]:.4f}, max={file_stats['max'][1]:.4f}, mean={file_stats['mean'][1]:.4f}")
                if len(file_stats['min']) > 2:
                    print(f"   Band 3 (Blue):  min={file_stats['min'][2]:.4f}, max={file_stats['max'][2]:.4f}, mean={file_stats['mean'][2]:.4f}")
                print()
        except Exception as e:
            print(f"   ⚠️ Error reading {fusion_file.name}: {e}\n")
    
    # Check consistency
    if len(stats_data) > 1:
        means_band1 = [s['mean'][0] for s in stats_data if not np.isnan(s['mean'][0])]
        if means_band1:
            mean_std = np.std(means_band1)
            print(f"📊 Temporal consistency: Band 1 mean std across dates = {mean_std:.4f}")
            if mean_std < 0.1:
                print("   ✅ Values are consistent across dates")
            else:
                print("   ⚠️  Values vary significantly (may indicate issues)")
    
    print("=" * 70)


### 5.2 RGB Comparison (S2 vs S3 vs Fusion)


In [ ]:
# 2. RGB Side-by-Side Comparison - Spatial quality verification

if not RASTERIO_AVAILABLE:
    print("⚠️ Install rasterio for visualization")
elif 'filtered_fusion' not in globals():
    print("❌ Variables not found. Run the data loading cell (Cell 5) first.")
elif not filtered_fusion:
    print("⚠️  No fusion files found!")
    print(f"   Cannot show fusion comparison without fusion results.")
    print(f"   💡 Run the EFAST fusion step first, then rerun this cell.")
    
    # Still show S2/S3 comparison if available
    if 's2_files' in globals() and s2_files and 's3_composites' in globals() and s3_composites:
        print(f"\n   Showing S2 vs S3 comparison instead...")
        try:
            s2_file = s2_files[0]
            s3_file = s3_composites[len(s3_composites) // 2]
            
            fig, axes = plt.subplots(1, 2, figsize=(14, 6))
            
            # S2
            with rasterio.open(s2_file) as src:
                red = src.read(3).astype(float)
                green = src.read(2).astype(float)
                blue = src.read(1).astype(float)
                rgb = np.dstack((red, green, blue))
                rgb[rgb == src.nodata] = np.nan
                rgb = np.clip(rgb / 3000.0, 0, 1)
                axes[0].imshow(rgb)
                axes[0].set_title('S2 Original (High-Res)', fontsize=12, fontweight='bold')
                axes[0].axis('off')
            
            # S3
            with rasterio.open(s3_file) as src:
                data = src.read(1).astype(float)
                data[data == src.nodata] = np.nan
                im = axes[1].imshow(data, cmap='RdYlGn', vmin=0, vmax=0.8)
                axes[1].set_title('S3 Low-Res', fontsize=12, fontweight='bold')
                axes[1].axis('off')
                plt.colorbar(im, ax=axes[1], fraction=0.046)
            
            plt.suptitle('S2 vs S3 Comparison (Fusion results not available)', 
                        fontsize=14, fontweight='bold')
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"   ⚠️ Error showing S2/S3 comparison: {e}")
else:
    print("=" * 70)
    print("2️⃣  RGB COMPARISON (S2 vs S3 vs Fusion)")
    print("=" * 70)
    
    # Find a fusion file and try to match with S2/S3
    fusion_file = filtered_fusion[len(filtered_fusion) // 2]  # Middle file
    fusion_date_str = fusion_file.stem.split('_')[1]
    fusion_date = datetime.strptime(fusion_date_str, '%Y%m%d')
    
    print(f"📅 Comparing date: {fusion_date.strftime('%Y-%m-%d')}\n")
    
    # Find closest S2 file
    s2_match = None
    min_diff = float('inf')
    for s2_file in s2_files:
        try:
            date_str = s2_file.stem.split('_')[2]
            s2_date = datetime.strptime(date_str, '%Y%m%d')
            diff = abs((s2_date - fusion_date).days)
            if diff < min_diff:
                min_diff = diff
                s2_match = s2_file
        except (IndexError, ValueError):
            continue
    
    # Find closest S3 composite
    s3_match = None
    min_diff_s3 = float('inf')
    for s3_file in s3_composites:
        try:
            if 'composite_' in s3_file.stem:
                date_str = s3_file.stem.split('composite_')[1]
                s3_date = datetime.strptime(date_str, '%Y-%m-%d')
                diff = abs((s3_date - fusion_date).days)
                if diff < min_diff_s3:
                    min_diff_s3 = diff
                    s3_match = s3_file
        except (ValueError, IndexError):
            continue
    
    # Create comparison figure
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # S2 (if available)
    if s2_match and min_diff <= 30:
        try:
            with rasterio.open(s2_match) as src:
                red = src.read(3).astype(float)
                green = src.read(2).astype(float)
                blue = src.read(1).astype(float)
                rgb = np.dstack((red, green, blue))
                rgb[rgb == src.nodata] = np.nan
                rgb = np.clip(rgb / 3000.0, 0, 1)
                
                axes[0].imshow(rgb)
                axes[0].set_title(f'S2 Original\n({min_diff} days from fusion)', 
                                 fontsize=12, fontweight='bold')
                axes[0].axis('off')
        except Exception as e:
            axes[0].text(0.5, 0.5, f'S2 not available\n({e})', 
                        ha='center', va='center', transform=axes[0].transAxes)
            axes[0].axis('off')
    else:
        axes[0].text(0.5, 0.5, 'S2 not available\n(no matching date)', 
                    ha='center', va='center', transform=axes[0].transAxes)
        axes[0].axis('off')
    
    # S3 (if available)
    if s3_match and min_diff_s3 <= 30:
        try:
            with rasterio.open(s3_match) as src:
                data = src.read(1).astype(float)
                data[data == src.nodata] = np.nan
                
                im = axes[1].imshow(data, cmap='RdYlGn', vmin=0, vmax=0.8)
                axes[1].set_title(f'S3 Low-Res\n({min_diff_s3} days from fusion)', 
                                 fontsize=12, fontweight='bold')
                axes[1].axis('off')
                plt.colorbar(im, ax=axes[1], fraction=0.046)
        except Exception as e:
            axes[1].text(0.5, 0.5, f'S3 not available\n({e})', 
                        ha='center', va='center', transform=axes[1].transAxes)
            axes[1].axis('off')
    else:
        axes[1].text(0.5, 0.5, 'S3 not available\n(no matching date)', 
                    ha='center', va='center', transform=axes[1].transAxes)
        axes[1].axis('off')
    
    # Fusion result
    try:
        with rasterio.open(fusion_file) as src:
            red = src.read(3).astype(float) if src.count >= 3 else src.read(1).astype(float)
            green = src.read(2).astype(float) if src.count >= 2 else src.read(1).astype(float)
            blue = src.read(1).astype(float)
            
            rgb = np.dstack((red, green, blue))
            rgb[rgb == src.nodata] = np.nan
            rgb = np.clip(rgb / 3000.0, 0, 1)
            
            axes[2].imshow(rgb)
            axes[2].set_title(f'Fusion Result\n(High-Res Daily)', 
                            fontsize=12, fontweight='bold')
            axes[2].axis('off')
    except Exception as e:
        axes[2].text(0.5, 0.5, f'Error loading fusion\n({e})', 
                    ha='center', va='center', transform=axes[2].transAxes)
        axes[2].axis('off')
    
    plt.suptitle(f'RGB Comparison - {sitename} ({season_year or "all seasons"})', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("=" * 70)


### 5.3 Time Series (Temporal Smoothness)


In [ ]:
# 3. Time Series Plot - Temporal smoothness verification

if not RASTERIO_AVAILABLE:
    print("⚠️ Install rasterio for visualization")
elif 'filtered_fusion' not in globals():
    print("❌ Variables not found. Run the data loading cell (Cell 5) first.")
elif not filtered_fusion:
    print("⚠️  No fusion files found!")
    print(f"   Cannot create fusion time series without fusion results.")
    print(f"   💡 Run the EFAST fusion step first, then rerun this cell.")
    
    # Show S3 time series as alternative
    if 's3_composites' in globals() and s3_composites:
        print(f"\n   Showing S3 composite time series instead...")
        try:
            dates_ts = []
            values = []
            
            for comp_file in s3_composites[:50]:  # Limit to first 50
                try:
                    with rasterio.open(comp_file) as src:
                        data = src.read(1).astype(float)
                        data[data == src.nodata] = np.nan
                        h, w = data.shape
                        center_val = data[h//2, w//2]
                        if not np.isnan(center_val):
                            if 'composite_' in comp_file.stem:
                                date_str = comp_file.stem.split('composite_')[1]
                                dates_ts.append(datetime.strptime(date_str, '%Y-%m-%d'))
                                values.append(center_val)
                except Exception as e:
                    continue
            
            if dates_ts and values:
                fig, ax = plt.subplots(figsize=(14, 6))
                ax.plot(dates_ts, values, marker='o', markersize=4, linewidth=2, 
                       color='green', alpha=0.7, label='S3 Composite')
                ax.set_xlabel('Date', fontsize=12, fontweight='bold')
                ax.set_ylabel('Reflectance (center pixel)', fontsize=12, fontweight='bold')
                ax.set_title(f'S3 Composite Time Series - {sitename if "sitename" in globals() else "unknown"}\n(Fusion results not available)', 
                           fontsize=13, fontweight='bold')
                ax.legend(loc='best', fontsize=10)
                ax.grid(True, alpha=0.3)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()
        except Exception as e:
            print(f"   ⚠️ Error creating S3 time series: {e}")
else:
    print("=" * 70)
    print("3️⃣  TIME SERIES (Temporal Smoothness)")
    print("=" * 70)
    
    # Extract time series from center pixel
    dates_ts = []
    values_band1 = []
    values_band2 = []
    values_band3 = []
    
    for fusion_file in filtered_fusion:
        try:
            with rasterio.open(fusion_file) as src:
                data = src.read()
                date_str = fusion_file.stem.split('_')[1]
                date_obj = datetime.strptime(date_str, '%Y%m%d')
                
                h, w = data.shape[1], data.shape[2]
                center_h, center_w = h // 2, w // 2
                
                # Extract center pixel values
                val1 = data[0, center_h, center_w] if not np.isnan(data[0, center_h, center_w]) else None
                val2 = data[1, center_h, center_w] if data.shape[0] > 1 and not np.isnan(data[1, center_h, center_w]) else None
                val3 = data[2, center_h, center_w] if data.shape[0] > 2 and not np.isnan(data[2, center_h, center_w]) else None
                
                if val1 is not None:
                    dates_ts.append(date_obj)
                    values_band1.append(float(val1))
                    values_band2.append(float(val2) if val2 is not None else np.nan)
                    values_band3.append(float(val3) if val3 is not None else np.nan)
        except Exception as e:
            continue
    
    if dates_ts and values_band1:
        try:
            fig, ax = plt.subplots(figsize=(14, 6))
            
            ax.plot(dates_ts, values_band1, marker='o', markersize=4, linewidth=2, 
                   label='Band 1 (Red)', color='red', alpha=0.7)
            if any(not np.isnan(v) for v in values_band2):
                ax.plot(dates_ts, values_band2, marker='s', markersize=4, linewidth=2, 
                       label='Band 2 (Green)', color='green', alpha=0.7)
            if any(not np.isnan(v) for v in values_band3):
                ax.plot(dates_ts, values_band3, marker='^', markersize=4, linewidth=2, 
                       label='Band 3 (Blue)', color='blue', alpha=0.7)
            
            ax.set_xlabel('Date', fontsize=12, fontweight='bold')
            ax.set_ylabel('Reflectance (center pixel)', fontsize=12, fontweight='bold')
            ax.set_title(f'Fusion Time Series - {sitename} ({season_year or "all seasons"})\nCenter Pixel Values', 
                       fontsize=13, fontweight='bold')
            ax.legend(loc='best', fontsize=10)
            ax.grid(True, alpha=0.3)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
            
            # Check for temporal smoothness
            if len(values_band1) > 1:
                diffs = np.abs(np.diff(values_band1))
                max_jump = np.max(diffs)
                mean_jump = np.mean(diffs)
                print(f"📊 Temporal smoothness (Band 1):")
                print(f"   Mean change between dates: {mean_jump:.4f}")
                print(f"   Max jump: {max_jump:.4f}")
                if max_jump < 0.2:
                    print("   ✅ Smooth temporal transitions")
                elif max_jump < 0.5:
                    print("   ⚠️  Some temporal variability (may be normal)")
                else:
                    print("   ⚠️  Large temporal jumps detected (check for artifacts)")
        except Exception as e:
            print(f"   ⚠️ Error creating time series: {e}")
    else:
        print("   ⚠️ Could not extract time series data")
    
    print("=" * 70)
    print("✅ Verification complete!")
    print("=" * 70)
